In [ ]:
# import cv2
# import os
# from tensorflow.keras.models import load_model
# import numpy as np
# from pygame import mixer
# import time

# # this is used to get beep sound (when person closes his eyes for more them 10sec)
# mixer.init()
# alarm_sound = mixer.Sound('alarm.wav')

# # this xml files are used to detect face , left eye , and right eye of a person.
# face_detection = cv2.CascadeClassifier('haar cascade files\haarcascade_frontalface_alt.xml')
# left_eye_detection = cv2.CascadeClassifier('haar cascade files\haarcascade_lefteye_2splits.xml')
# right_eye_detection = cv2.CascadeClassifier('haar cascade files\haarcascade_righteye_2splits.xml')

# labels =['Close','Open']

# # load the model, that we have created
# model1 = load_model('models/eye-drowsiness.h5')
# model12 = load_model('models/mouth-drowsiness.h5')
# path = os.getcwd()

# # to capture each frame
# capture = cv2.VideoCapture(0)

# #check if the webcam is opened correctly
# if capture.isOpened():
#     capture = cv2.VideoCapture(0)
# if not capture.isOpened():
#     raise IOError("Cannot open webcam")

# font = cv2.FONT_HERSHEY_COMPLEX_SMALL

# #declaring variables 
# counter = 0
# time = 0
# thick = 2
# right_eye_pred=[99]
# left_eye_pred=[99]

# while(True):
#     ret, frame = capture.read()
#     height,width = frame.shape[:2] 

#     #convert the captured image to grey color:
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     #perform detection(this will return x,y coordinates , height , width of the boundary boxes object)
#     faces = face_detection.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
#     left_eye = left_eye_detection.detectMultiScale(gray)
#     right_eye =   right_eye_detection.detectMultiScale(gray)

#     cv2.rectangle(frame, (0,height-50) , (100,height) , (0,0,0) , thickness=cv2.FILLED )
#     cv2.rectangle(frame, (290,height-50) , (540,height) , (0,0,0) , thickness=cv2.FILLED )

#     #iterating over faces and drawing boundary boxes for each face:
#     for (x,y,w,h) in faces:
#         cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )
        
#     #iterating over right eye:
#     for (x,y,w,h) in right_eye:
#         #pull out the right eye image from the frame:
#         right_one=frame[y:y+h,x:x+w]
#         counter += 1
#         right_one = cv2.cvtColor(right_one,cv2.COLOR_BGR2GRAY)
#         right_one = cv2.resize(right_one,(24,24))
#         right_one = right_one/255
#         right_one =  right_one.reshape(24,24,-1)
#         right_one = np.expand_dims(right_one,axis=0)
#         print(right_one.shape)
#         right_eye_pred = model.predict(right_one)
#         print(f'Right: {right_eye_pred}')
#         if(right_eye_pred[0][0] > 0.5):
#             labels = 'Open' 
#         if(right_eye_pred[0][0] < 0.5):
#             labels = 'Closed'
#         break

#     #iterating over left eye:
#     for (x,y,w,h) in left_eye:
#         #pull out the left eye image from the frame:
#         left_one=frame[y:y+h,x:x+w]
#         counter += 1
#         left_one = cv2.cvtColor(left_one,cv2.COLOR_BGR2GRAY)  
#         left_one = cv2.resize(left_one,(24,24))
#         left_one = left_one/255
#         left_one = left_one.reshape(24,24,-1)
#         left_one = np.expand_dims(left_one,axis=0)
#         left_eye_pred = model.predict(left_one)
#         print(f'Left: {left_eye_pred[0,0]}')
#         if(left_eye_pred[0][0] > 0.5):
#             labels ='Open'   
#         if(left_eye_pred[0][0] < 0.5):
#             labels ='Closed'
#         break

#     if(right_eye_pred[0][0] < 0.5 and left_eye_pred[0][0] < 0.5):
#         time += 1
#         cv2.putText(frame,"Inactive",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
#     # if(right_eye_pred[0]==1 or left_eye_pred[0]==1):
#     else:
#         time -= 1
#         cv2.putText(frame,"Active",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)

#     if(time<0):
#         time=0   
#     cv2.putText(frame,'Wake up Time !!:'+str(time),(300,height-20), font, 1,(0,0,255),1,cv2.LINE_AA)
#     if(time>10):
#         #person is feeling dazzi we will alert :
#         cv2.imwrite(os.path.join(path,'image.jpg'),frame)
#         try:
#             alarm_sound.play()
            
#         except:  # isplaying = False
#             pass
#         if(thick < 16):
#             thick = thick+2
#         else:
#             thick=thick-2
#             if(thick<2):
#                 thick=2
#         cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thick)
#     cv2.imshow('frame',frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
    
# capture.release()
# cv2.destroyAllWindows()

In [5]:
import cv2
# import tensorflow
from tensorflow.keras.models import load_model
import numpy as np
# from datetime import datetime
# from pygame import mixer

# drowsy_detected = False
# start_time = None
# elapsed_time = 0
is_drowsy = False
is_drowsy2 = False
is_drowsy3 = False
numbers = 0
drowsy_duration = 5
# mixer.init()
# alarm_sound = mixer.Sound('alarm.wav')

model1 = load_model('models/eye-drowsiness6.h5')
model2 = load_model('models/mouth-drowsiness3.h5')

def preprocess_image(face_image):
    processed_image = cv2.resize(face_image, (150, 150))  # Ubah ukuran gambar
    processed_image = processed_image / 255.0  # Normalisasi
    return processed_image

def detect_mouth(face_roi):
    processed_image = preprocess_image(face_roi)  
    prediction = model2.predict(np.expand_dims(processed_image, axis=0))
    return prediction[0][0] > 0.5

def detect_eye(face_roi):
    processed_image = preprocess_image(face_roi)  
    prediction = model1.predict(np.expand_dims(processed_image, axis=0))
    return prediction[0][0] > 0.5

face_detection = cv2.CascadeClassifier('haar cascade files\haarcascade_frontalface_alt.xml')
# face_detection = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
left_eye_detection = cv2.CascadeClassifier('haar cascade files\haarcascade_lefteye_2splits.xml')
right_eye_detection = cv2.CascadeClassifier('haar cascade files\haarcascade_righteye_2splits.xml')

# Mulai akses webcam
cap = cv2.VideoCapture(0)  # Gunakan 0 jika webcam default
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # faces = face_detection.detectMultiScale(gray, 1.1, 4)
    faces = face_detection.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    left_eye = left_eye_detection.detectMultiScale(gray)
    right_eye = right_eye_detection.detectMultiScale(gray)
        
    #iterating over right eye:
    for (x,y,w,h) in right_eye:
        right_one=frame[y:y+h,x:x+w]
        is_drowsy2 = detect_eye(right_one)
        if (is_drowsy2 == True):
            cv2.rectangle(frame, (x, y), (x + w, y + h), (85, 225, 80), 2)
            numbers = 0
        else:
            numbers = numbers + 1
            if numbers >= drowsy_duration:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            elif numbers < drowsy_duration:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (25, 227, 255), 2)

    #iterating over left eye:
    for (x,y,w,h) in left_eye:
        left_one=frame[y:y+h,x:x+w]
        is_drowsy3 = detect_eye(left_one)
        if (is_drowsy3 == True):
            cv2.rectangle(frame, (x, y), (x + w, y + h), (85, 225, 80), 2)
            numbers = 0
        else:
            numbers = numbers + 1
            if numbers >= drowsy_duration:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            elif numbers < drowsy_duration:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (25, 227, 255), 2)
    
    for (x,y,w,h) in faces:
        face_roi = frame[y:y + h, x:x + w]  
        is_drowsy = detect_mouth(face_roi)
        if (is_drowsy == True):
            if numbers >= drowsy_duration:
                cv2.putText(frame, '!!!Alert, Wake Up', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 31, 255), 2)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            elif numbers < drowsy_duration:
                cv2.putText(frame, 'Little Drowsiness!', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (25, 227, 255), 2)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (25, 227, 255), 2)
        else:
            cv2.putText(frame, 'Awake', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (85, 225, 80), 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (85, 225, 80), 2)
    
    if (is_drowsy and is_drowsy2 and is_drowsy3):
        numbers = numbers + 1
        # if numbers >= drowsy_duration:
            # bunyikan alarm
    else:
        numbers = 0
    
    cv2.imshow('Drowsiness Detection', frame)

    # Tekan 'q' untuk keluar
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 112ms/step
